In [5]:
# Install dependencies
!pip install langchain langchain-community transformers faiss-cpu sentence-transformers streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [6]:
# Create a simple text corpus
corpus = """
Document 1: Artificial Intelligence (AI) is the simulation of human intelligence in machines. It includes tasks like reasoning, learning, and problem-solving.
Document 2: Machine Learning (ML) is a subset of AI that focuses on algorithms that learn from data. Popular ML models include neural networks and decision trees.
Document 3: Transformers are a type of ML model used in natural language processing. They rely on attention mechanisms to process sequences efficiently.
"""

# Save corpus to a file
with open("corpus.txt", "w") as f:
    f.write(corpus)

# Split corpus into documents
documents = corpus.strip().split("\n\n")

In [7]:
from langchain.docstore.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Convert text to LangChain Document objects
docs = [Document(page_content=doc) for doc in documents]

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vector_store = FAISS.from_documents(docs, embeddings)

/tmp/ipython-input-7-1591848543.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models o

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load lightweight LLM (GPT-2)
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create text generation pipeline with updated parameters
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200,  # Increase max_length to handle longer inputs
    max_new_tokens=50,  # Limit new tokens generated to avoid excessive output
    truncation=True
)

# Wrap in LangChain's LLM
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

# Set up conversational memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create RAG chain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    memory=memory
)

Device set to use cpu


In [11]:
# Test query
query = "What are Transformers in machine learning?"
result = rag_chain.invoke({"question": query})  # Use .invoke() instead of direct call

# Print response
print("Question:", query)
print("Answer:", result["answer"])

# Test follow-up question
follow_up = "How do they differ from neural networks?"
result = rag_chain.invoke({"question": follow_up})  # Use .invoke()
print("Follow-up Question:", follow_up)
print("Answer:", result["answer"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What are Transformers in machine learning?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Document 1: Artificial Intelligence (AI) is the simulation of human intelligence in machines. It includes tasks like reasoning, learning, and problem-solving.
Document 2: Machine Learning (ML) is a subset of AI that focuses on algorithms that learn from data. Popular ML models include neural networks and decision trees.
Document 3: Transformers are a type of ML model used in natural language processing. They rely on attention mechanisms to process sequences efficiently.

Question: What are Transformers in machine learning?
Helpful Answer: Transformers are a subset of AI that focuses on algorithms that learn from data. Popular ML models include neural nets and decision trees.

Question: What are Transformers in machine learning?

Helpful Answer: Transformers are a s

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Follow-up Question: How do they differ from neural networks?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Document 1: Artificial Intelligence (AI) is the simulation of human intelligence in machines. It includes tasks like reasoning, learning, and problem-solving.
Document 2: Machine Learning (ML) is a subset of AI that focuses on algorithms that learn from data. Popular ML models include neural networks and decision trees.
Document 3: Transformers are a type of ML model used in natural language processing. They rely on attention mechanisms to process sequences efficiently.

Question: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What are Transformers in machine learning?
Assistant: Use the following pieces of context to answer the question